# Faces recognition example
The dataset used in this example is a preprocessed excerpt of the
"Labeled Faces in the Wild", aka LFW_:

  http://vis-www.cs.umass.edu/lfw/lfw-funneled.tgz (233MB)

  .. _LFW: http://vis-www.cs.umass.edu/lfw/

  original source: http://scikit-learn.org/stable/auto_examples/applications/face_recognition.html

In [8]:
from time import time
import logging
import pylab as pl
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(message)s')

In [2]:
# Download the data, if not already on disk and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

2016-12-27 14:30:08,444 Loading LFW people faces from /home/rad/scikit_learn_data/lfw_home


In [3]:
# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
np.random.seed(42)

In [4]:
# for machine learning we use the data directly (as relative pixel
# position info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]

In [5]:
# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]

print "Total dataset size:"
print "n_samples: %d" % n_samples
print "n_features: %d" % n_features
print "n_classes: %d" % n_classes

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [ ]:
# Split into a training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [19]:
# Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
# dataset): unsupervised feature extraction / dimensionality reduction
def extract_faces(X_train, X_test, y_train, y_test, n_components = 150):
    print "Extracting the top %d eigenfaces from %d faces" % (n_components, X_train.shape[0])
    t0 = time()
    pca = PCA(svd_solver='randomized',n_components=n_components, whiten=True).fit(X_train)
    print "done in %0.3fs" % (time() - t0)

    eigenfaces = pca.components_.reshape((n_components, h, w))

    print "Projecting the input data on the eigenfaces orthonormal basis"
    t0 = time()
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    print "done in %0.3fs" % (time() - t0)

    # Train a SVM classification model

    print "Fitting the classifier to the training set"
    t0 = time()
    param_grid = {
             'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1],
              }
    # for sklearn version 0.16 or prior, the class_weight parameter value is 'auto'
    clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
    clf = clf.fit(X_train_pca, y_train)
    print "done in %0.3fs" % (time() - t0)
#     print "Best estimator found by grid search:"
#     print clf.best_estimator_
    return clf, X_train_pca, X_test_pca

In [17]:
#How much variance is explained by the first two principal components?
pca.explained_variance_ratio_[:2]

array([ 0.18880161,  0.14976933])

In [20]:
def predict_faces(clf, X_test_pca, y_test):
    print "Predicting the people names on the testing set"
    t0 = time()
    y_pred = clf.predict(X_test_pca)
    print "done in %0.3fs" % (time() - t0)

    print classification_report(y_test, y_pred, target_names=target_names)
    print confusion_matrix(y_test, y_pred, labels=range(n_classes))

In [21]:
for n_components in [10, 15, 25, 50, 100, 250]:
    clf, X_train_pca, X_test_pca = extract_faces(X_train, X_test, y_train, y_test, n_components = n_components)
    predict_faces(clf, X_test_pca, y_test)

Extracting the top 10 eigenfaces from 966 faces
done in 0.201s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.041s
Fitting the classifier to the training set
done in 50.101s
Predicting the people names on the testing set
done in 0.014s
                   precision    recall  f1-score   support

     Ariel Sharon       0.16      0.23      0.19        13
     Colin Powell       0.47      0.52      0.49        60
  Donald Rumsfeld       0.30      0.37      0.33        27
    George W Bush       0.59      0.53      0.56       146
Gerhard Schroeder       0.12      0.12      0.12        25
      Hugo Chavez       0.27      0.27      0.27        15
       Tony Blair       0.27      0.22      0.24        36

      avg / total       0.44      0.43      0.43       322

[[ 3  5  0  5  0  0  0]
 [ 5 31  6 13  3  0  2]
 [ 3  5 10  6  2  0  1]
 [ 5 18 10 78 11 10 14]
 [ 1  2  4 11  3  1  3]
 [ 0  1  0  6  2  4  2]
 [ 2  4  3 14  5  0  8]]
Extracting the top 15 eigenfaces fro